In [1]:
# Bibliotecas Necessárias
!pip install -q transformers sentence-transformers langchain langchain-community faiss-cpu # Ensure these are installed
!pip install -q accelerate bitsandbytes # For loading large models efficiently (e.g., Gemma)
!pip install -q google-generativeai # If you plan to use Google Gemini API

In [2]:
# Imports Necessários
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# LLM
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [3]:
# Configuração Inicial
diretorio_projeto = "Nuvem-Desafio-Tecnico-IA"
diretorio_faiss = os.path.join(diretorio_projeto, "faiss_index")

os.makedirs(diretorio_faiss, exist_ok=True)

modelo = "sentence-transformers/all-MiniLM-L6-v2"

modelo_HF = "google/gemma-2b-it"

tam_chunk = 1000
overlap = 100

In [4]:
# Carrega FAISS
kwarg = 'cuda' if torch.cuda.is_available() else 'cpu'
embeddings = HuggingFaceEmbeddings(model_name=modelo,
                                   model_kwargs={'device': kwarg})

if not os.path.exists(diretorio_faiss):
    print(f"Erro: Diretório de indices FAISS '{diretorio_faiss}' não encontrado.")

else:
    vectorstore = FAISS.load_local(diretorio_faiss, embeddings, allow_dangerous_deserialization=True)
    print(f"Número de documentos em FAISS index: {len(vectorstore.docstore._dict)}")


/tmp/ipython-input-4-775735882.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=modelo,


Número de documentos em FAISS index: 230


In [5]:
# Inicializa o LLM
llm = None

print(f"\nInicializando HuggingFace LLM: {modelo_HF}...")

tokenizer = AutoTokenizer.from_pretrained(modelo_HF)

model = AutoModelForCausalLM.from_pretrained(
    modelo_HF,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Cria a pipeline da Hugging Face para geração de texto
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500, # Max tokens to generate in response
    do_sample=True,
    temperature=0.7, # Controls randomness: lower is more deterministic
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.1,
    trust_remote_code=True,
    model_kwargs={"torch_dtype": torch.bfloat16} # Ensure consistent dtype
)

llm = HuggingFacePipeline(pipeline=pipe)
print(f"HuggingFace LLM '{modelo_HF}' inicializado.")


Inicializando HuggingFace LLM: google/gemma-2b-it...


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0


HuggingFace LLM 'google/gemma-2b-it' inicializado.


/tmp/ipython-input-5-4238876901.py:29: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [6]:
# Setando RetrievalQA Chain
template =  """Você é um assistente de IA. Use SOMENTE o contexto fornecido para responder à pergunta.
Se a resposta NÃO ESTIVER no contexto fornecido, diga "Não sei" e nada mais.
Não tente inventar uma resposta.
Mantenha a resposta concisa e relevante ao contexto.

Contexto:
{context}

Pergunta: {question}

Resposta:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# k é o número de chunks recuperados
k = 5

# Create the RAG chain
# We use 'stuff' chain type, which stuffs all retrieved documents into the prompt.
# For many documents, consider 'map_reduce' or 'refine' for larger contexts.
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": k}),
    chain_type="stuff",
    return_source_documents=True, # Retorna os documentos uados
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

print("RetrievalQA chain set up.")

RetrievalQA chain set up.


In [7]:
# Função de Fazer a Pergunta

def responder_pergunta(pergunta_do_usuario):
    """
    Recebe uma pergunta em linguagem natural, busca informações relevantes
    e gera uma resposta usando a LLM.
    """
    print(f"\n--- Processando pergunta: '{pergunta_do_usuario}' ---")
    try:
        result = qa_chain.invoke({"query": pergunta_do_usuario})

        resposta = result["result"]
        source_documents = result["source_documents"]

        print("\nResposta da LLM:")
        print(resposta)

        print("\nDocumentos de origem recuperados:")
        for doc in source_documents:
            print(f"- Fonte: {doc.metadata.get('Fonte', 'N/A')}")
            print(f"  Trecho (início): {doc.page_content[:200]}...")
            print("---")

    except Exception as e:
        print(f"Ocorreu um erro ao processar a pergunta: {e}")

In [8]:
# --- Exemplos de Perguntas ---

# Pergunta 1: Sobre o Código de Obras
responder_pergunta("Quais são as disposições preliminares do Código de Obras?")

# Pergunta 2: Sobre a Tabela (assume que a tabela foi OCR'd bem o suficiente para extrair valores)
responder_pergunta("Qual o preço do Extintor de 1 kg na Tabela de Custos?")

# Pergunta 3: Uma pergunta que pode não ter resposta no contexto dos seus documentos
responder_pergunta("Qual a capital da França?")

# Pergunta 4: Outra pergunta sobre o Código de Obras
responder_pergunta("Quais são os tipos de edificações segundo o Código de Obras?")



--- Processando pergunta: 'Quais são as disposições preliminares do Código de Obras?' ---

Resposta da LLM:
Você é um assistente de IA. Use SOMENTE o contexto fornecido para responder à pergunta.
Se a resposta NÃO ESTIVER no contexto fornecido, diga "Não sei" e nada mais.
Não tente inventar uma resposta.
Mantenha a resposta concisa e relevante ao contexto.

Contexto:
Município de Eusébio e adota outras providências.
A Câmara Municipal de Eusébio decreta e eu sanciono a seguinte Lei:
CAPÍTULO I
DISPOSIÇÕES PRELIMINARES
Art. 1o. Fica instituído o Código de Obras, Edificações e Posturas do Município de Eusébio, o qual
estabelece normas para a elaboração de projetos e execução de obras e instalações, em seus aspectos
técnicos estruturais e funcionais, bem como estabelece medidas de polícia administrativa de
competência do Município, no que diz respeito à ordem pública, higiene, instalação e funcionamento de
equipamentos e atividades quando do uso dos espaços públicos e privados.
§1o. Todo